# ML cube Platform SDK - Production Routine

In this notebook, you will see how to set up a production routine. A production routine is composed of sending production data, check the model status and then ask the retraining report if a drift was detected in the data.

**Requirements**:

1. API Key with roles `COMPANY_ADMIN` or `PROJECT_ADMIN`
2. Id of the task
3. Id and version of the model

**User Input**

You will need to provide some values for variables and names to ensure the notebook runs correctly. Whenever you see the comment `# TO COMPLETE`, make sure to fill the empty string accordingly.

**Define entities ids**

Specify here the ids of the entities that are required to work on this notebook.

In [ ]:
# TO COMPLETE
task_id = ""
model_id = ""
model_version = ""

**Imports**

In [ ]:
import logging
logger = logging.getLogger("platform_tutorial")

In [ ]:
from ml3_platform_sdk.client import ML3PlatformClient
from ml3_platform_sdk import enums as ml3_enums
from ml3_platform_sdk import models as ml3_models

**Instantiate the Client**

To interact with ML cube Platform, you need to instantiate the client. You will then use its methods to perform requests.
Please, insert the api key we provided you to instantiate the client.

In [ ]:
API_KEY = ""  # TO COMPLETE
URL = "https://api.platform.mlcube.com"
ml3_client = ML3PlatformClient(api_key=API_KEY, url=URL)

**Create Detection Automation Rule**

Production data are analysed by our monitoring algorithms to detect drifts.
You can setup automation rules to receives notifications in your desired communication channel and to retrain automatically your model.

In [ ]:
# TO COMPLETE
rule_id = ml3_client.create_detection_event_rule(
    name='rule1',
    task_id=task_id,
    model_id=model_id,
    severity=ml3_enums.DetectionEventSeverity.MEDIUM,
    detection_event_type=ml3_enums.DetectionEventType.DRIFT_ON,
    monitoring_target=ml3_enums.MonitoringTarget.ERROR,
    actions=[
        ml3_models.DiscordNotificationAction(webhook='discord_webhook'),
        ml3_models.SlackNotificationAction(webhook='slack_webhook', channel='slack_channel'),
        ml3_models.RetrainAction()
    ],
)
logger.info(f"Created event rule with id {rule_id}")

**Add production data**

Production data are uploaded to the ML Cube Platform in the same way as historical data
However, in this scenario, it is common to have data arrive at different times. For example, your model may receive input data and generate predictions, but the actual targets might only become available after a delay.
Therefore, you can upload data as soon as you receive it, rather than all at once, and the ML Cube Platform will begin monitoring as soon as the data is available.

In [ ]:
# TO COMPLETE
job_id = ml3_client.add_production_data(
    task_id=task_id,
    inputs=ml3_models.TabularData(
        source=ml3_models.LocalDataSource(
            file_type=ml3_enums.FileType.CSV,
            file_path="path/to/file.csv",
            is_folder=False,
            folder_type=None,
        )
    ),
    target=None,
    predictions=[
        (
            model_id,
            ml3_models.TabularData(
                source=ml3_models.LocalDataSource(
                    file_type=ml3_enums.FileType.CSV,
                    file_path="path/to/file.csv",
                    is_folder=False,
                    folder_type=None,
                )
            ),
        )
    ],
)
logger.info(f'Add production input and predictions data job {job_id}')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Add production input and predictions data job {job_id} terminated')

In [ ]:
job_id = ml3_client.add_production_data(
    task_id=task_id,
    inputs=None,
    target=ml3_models.TabularData(
        source=ml3_models.LocalDataSource(
            file_type=ml3_enums.FileType.CSV,
            file_path="path/to/file.csv",
            is_folder=False,
            folder_type=None,
        )
    ),
    predictions=None,
)
logger.info(f'Add production target data job {job_id} started')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Add production target data job {job_id} terminated')

**Check the batches uploaded**

To check the details of each batch uploaded you can use the `get_data_batch_list` method, it will return a list of the data batches uploaded up to this point.

In [ ]:
logger.info(f'Get data batches')
data_batches = ml3_client.get_data_batch_list(task_id=task_id)
logger.info(f'Number of data batches loaded: {len(data_batches)}')

**Check model status and ask retraining report**

To monitor the status of your model, you can use the `get_monitoring_status` method and specify the monitoring status `ERROR`.

In [ ]:
logger.info(f'Read model by id')
status = ml3_client.get_monitoring_status(task_id=task_id, monitoring_target=ml3_enums.MonitoringTarget.ERROR)
logger.info(f'Status: {status}')

When a drift is detected in a certain monitoring target, its status changes from `OK` to `DRIFT`. This is the moment when you really need ML cube Platform!
The best action to do is to ask the computation of the retraining report, that contains all the information to handle the drift and retrain your model.

In [ ]:
logger.info(f'Compute retraining report')
job_id = ml3_client.compute_retraining_report(model_id=model_id)
logger.info(f'Job created, id {job_id}')

ml3_client.show_jobs()

logger.info(f'Waiting job completion')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Job completed')

logger.info(f'Get retraining report')
retraining_report = ml3_client.get_retraining_report(model_id=model_id)

logger.info(f'Retraining Report: {retraining_report}')

**Retrain your model and upload the update to the ML cube Platform**

When you receive the retraining report, you can retrain the model with the importance weights we provided you. Afterward, you need to notify the creation of the new version of the model to the ML cube Platform.

In [ ]:
logger.info("API - Show suggestions")
ml3_client.show_suggestions(model_id=model_id, model_version=model_version)

logger.info(f'API - Update model version from suggestion id')
new_model_version = 'v0.0.2'
job_id = ml3_client.update_model_version_by_suggestion_id(
    model_id=model_id,
    new_model_version=new_model_version,
    suggestion_id=retraining_report.suggestion.suggestion_id,
)

logger.info(f'Job created, id {job_id}')

logger.info(f'Waiting job completion')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Job completed')

**Congratulations!**

In this notebook, you learned how to create a detection automation rule, add production data, even at different times, check the model status and ask a retraining report if a drift was detected.